<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.models import Word2Vec

In [2]:
max_seq_length = 100

In [3]:
ratings1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/ratings.csv')
#print(len(ratings1['movieId'].unique()))
ratings = ratings1.iloc[0:1000,:]
#print(len(ratings['movieId'].unique()))
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


**Sorting the dataframe**

In [4]:
ratings.sort_values(by=['userId','timestamp'],inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
ratings.head(10)

,userId,movieId,rating,timestamp
36,1,5952,4.0,1147868053
13,1,2012,2.5,1147868068
12,1,2011,2.5,1147868079
11,1,1653,4.0,1147868097
9,1,1250,4.0,1147868414
40,1,6539,3.5,1147868461
39,1,6377,4.0,1147868469
21,1,3448,4.0,1147868480
5,1,1088,4.0,1147868495
4,1,899,3.5,1147868510


In [6]:
users_total = len(ratings['userId'].unique())
print(users_total)

4


**Selecting the most recent movies rated by each user and padding if necessary**

In [7]:
movie_list = []
for i in range(users_total):
  list1 = []
  list1 = ratings.loc[ratings['userId'] ==(i+1),['movieId']]['movieId'].tolist()
  if len(list1)>max_seq_length:
    list1 = list1[(len(list1)-max_seq_length):]
  elif len(list1)<max_seq_length:
    for j in range((max_seq_length-len(list1))):
      list1.append(0)
  movie_list.append(list1)
#print(movie_list)

**Selecting the most recent ratings of each user and padding if necessary**

In [8]:
rating_list =[]
for i in range(users_total):
  list2 = []
  list2 = ratings.loc[ratings['userId'] ==(i+1),['rating']]['rating'].tolist()
  if len(list2)>max_seq_length:
    list2 = list2[(len(list2)-max_seq_length):]
  elif len(list2)<max_seq_length:
    for j in range((max_seq_length-len(list2))):
      list2.append(0)
  rating_list.append(list2)
#print(rating_list)

In [9]:
movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(users_total)])
movies_transpose.index.names = ['userId']
print(movies_transpose)

            0     1     2       3     4   ...     95      96     97    98    99
userId                                    ...                                  
1         5952  2012  2011    1653  1250  ...      0       0      0     0     0
2         4306  8368  8360    3793  4995  ...   2987    1293   7090  1136  2150
3       176371  3968  8983  102407  6548  ...  88125  106489  81834  4262  1732
4         8665  8961  4993    1580  5444  ...      0       0      0     0     0

[4 rows x 100 columns]


In [10]:
ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(users_total)])
ratings_transpose.index.names = ['userId']
print(ratings_transpose)

         0    1    2    3    4    5    6   ...   93   94   95   96   97   98   99
userId                                     ...                                   
1       4.0  2.5  2.5  4.0  4.0  3.5  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2       4.5  5.0  4.0  5.0  5.0  4.0  5.0  ...  0.5  3.0  0.5  5.0  2.0  5.0  5.0
3       4.5  3.0  4.0  4.0  3.0  4.0  3.5  ...  2.5  4.0  3.5  3.5  3.5  4.5  4.5
4       3.5  5.0  4.5  4.5  4.0  4.0  4.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[4 rows x 100 columns]


In [11]:
# Select features from original dataset to form a new dataframe 
df1 = movies_transpose.iloc[:]# For each row, combine all the columns into one column
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']]

In [12]:
#list of lists format of the ratings transpose matrix
print(sent)

[['5952', '2012', '2011', '1653', '1250', '6539', '6377', '3448', '1088', '899', '4308', '2161', '6711', '3949', '8360', '5878', '306', '1175', '307', '1237', '7327', '8154', '7234', '2843', '4144', '7365', '2068', '4422', '4973', '6016', '8873', '2692', '27721', '7323', '6954', '8014', '7939', '6370', '8973', '4703', '31956', '5147', '8786', '1260', '2351', '7940', '7209', '8685', '7820', '7937', '7938', '8405', '4325', '2632', '1217', '8729', '5912', '5767', '665', '2573', '27266', '8327', '32591', '5269', '3569', '27193', '5684', '7318', '296', '7361', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['4306', '8368', '8360', '3793', '4995', '4963', '8636', '2355', '8665', '2571', '589', '33493', '7153', '1873', '1246', '1584', '3994', '2139', '31923', '4720', '2294', '2745', '2138', '858', '33660', '2268', '2501', '6539', '1270', '6565', '4535', '1198', '1302', '349', '3098', '1907'

In [13]:
model = Word2Vec(sent,min_count=1,size= 50,workers=3, window =3, sg = 1)

In [14]:
#user_vec = []
#for i in range(users_total):
#  a = np.zeros((50,))
#  for j in range(max_seq_length):
#    a = a+ model[sent[i][j]]
#  user_vec.append(a)

In [15]:
#user_vec

In [16]:
model.save('wordtovec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [17]:
corpus = gensim.matutils.Dense2Corpus(np.array(sent))
print(list(corpus))
total_topics = 10

[[(0, 5952.0), (1, 4306.0), (2, 176371.0), (3, 8665.0)], [(0, 2012.0), (1, 8368.0), (2, 3968.0), (3, 8961.0)], [(0, 2011.0), (1, 8360.0), (2, 8983.0), (3, 4993.0)], [(0, 1653.0), (1, 3793.0), (2, 102407.0), (3, 1580.0)], [(0, 1250.0), (1, 4995.0), (2, 6548.0), (3, 5444.0)], [(0, 6539.0), (1, 4963.0), (2, 4448.0), (3, 7153.0)], [(0, 6377.0), (1, 8636.0), (2, 103141.0), (3, 6539.0)], [(0, 3448.0), (1, 2355.0), (2, 139644.0), (3, 33794.0)], [(0, 1088.0), (1, 8665.0), (2, 187593.0), (3, 6874.0)], [(0, 899.0), (1, 2571.0), (2, 135143.0), (3, 1210.0)], [(0, 4308.0), (1, 589.0), (2, 90866.0), (3, 1610.0)], [(0, 2161.0), (1, 33493.0), (2, 86911.0), (3, 7438.0)], [(0, 6711.0), (1, 7153.0), (2, 27660.0), (3, 780.0)], [(0, 3949.0), (1, 1873.0), (2, 171763.0), (3, 34405.0)], [(0, 8360.0), (1, 1246.0), (2, 8861.0), (3, 1214.0)], [(0, 5878.0), (1, 1584.0), (2, 143385.0), (3, 2028.0)], [(0, 306.0), (1, 3994.0), (2, 112138.0), (3, 2985.0)], [(0, 1175.0), (1, 2139.0), (2, 5313.0), (3, 1291.0)], [(0, 30

In [18]:
lsi = models.LsiModel(corpus, num_topics=total_topics)
index = similarities.MatrixSimilarity(lsi[corpus])
lsi_data = lsi[corpus]

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
#list(lsi_data)

In [20]:
lsi_topics = lsi.print_topics()
for topic in lsi_topics:
  print(topic)

(0, '0.996*"2" + 0.057*"3" + 0.047*"1" + 0.040*"0"')
(1, '0.965*"3" + 0.252*"0" + -0.063*"2" + -0.045*"1"')
(2, '0.995*"1" + -0.063*"0" + 0.060*"3" + -0.048*"2"')
(3, '-0.965*"0" + 0.251*"3" + -0.075*"1" + 0.028*"2"')


In [21]:
lsi.save('lsimodel')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [22]:
tfidf = models.TfidfModel(corpus)
tfidf_data = tfidf[corpus]
print(list(tfidf_data))

[[(0, 0.9186551661069456), (3, 0.39506035714181714)], [(0, 0.6051297393815158), (3, 0.7961268733788974)], [(0, 0.8063703476896577), (3, 0.5914109082244431)], [(0, 0.962374247973125), (3, 0.2717274495485543)], [(0, 0.6137037357667979), (3, 0.7895363985946919)], [(0, 0.9515547532155332), (3, 0.30747935155539446)], [(0, 0.9570588367417666), (3, 0.28989374435212695)], [(0, 0.3264742182059674), (3, 0.945206107072316)], [(0, 0.4722897251747432), (3, 0.8814433705544366)], [(0, 0.9292482308246142), (3, 0.36945598588915124)], [(0, 0.9939614418421958), (3, 0.10972990536304723)], [(0, 0.7012159079509601), (3, 0.712948981650518)], [(0, 0.9994111406646993), (3, 0.03431285349959301)], [(0, 0.36218154888126436), (3, 0.9321075719303907)], [(0, 0.9990812305440113), (3, 0.042856677130453866)], [(0, 0.9948466448420678), (3, 0.10139109056756679)], [(0, 0.32785284841342377), (3, 0.9447288022428472)], [(0, 0.9511572929897252), (3, 0.3087066633431454)], [(0, 0.9701030218311882), (3, 0.2426934837073241)], [(0

In [23]:
#lsi_list = np.zeros((len(lsi_data), len(lsi_topics)), dtype=np.float64)
#for i in range(len(corpus)):
#    for j in range(len(lsi_topics)):
#        lsi_list[i][j]= list(lsi_data)[i][j][1]

tfidf_token= np.zeros((len(tfidf_data), 350), dtype=np.float64)
tfidf_vals= np.zeros((len(tfidf_data), 350), dtype=np.float64)

for i in range(len(corpus)):
    for k in range(len(list(tfidf_data)[i])):
        tfidf_token[i][k]=(list(tfidf_data))[i][k][0]
        tfidf_vals[i][k]=(list(tfidf_data))[i][k][1]

tfidf_list=list(tfidf_data)

In [24]:
tfidf_list[1]

[(0, 0.6051297393815158), (3, 0.7961268733788974)]